In [1]:
# !pip install faiss-gpu openai faiss-cpu langchain langchain_community pypdf sentence-transformers tf-keras

In [1]:
from transformers import pipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

2024-12-21 09:03:20.885681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734771800.900073    3708 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734771800.904563    3708 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 09:03:20.918681: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
pdf_files = ["Кртакое_руководство_по_разбрасователю_Амазон_ZG_TS_7501.pdf", "Руководство_по_бортовому_компьютеу_Аматрон-1.pdf",
             "Руководство_по_разбрасователю_Амазон_ZG_TS_7501-2.pdf", "Руководство по эксплуатации ISO BUS-1.pdf"]
documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents.extend(loader.load())
# Разделение текста на части для лучшего индексирования
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4092,
    chunk_overlap=2048
)
split_docs = text_splitter.split_documents(documents)

In [5]:
model_name = "MTSAIR/Cotype-Nano"

In [6]:
embeddings = HuggingFaceEmbeddings(model_name=model_name)
faiss_index = FAISS.from_documents(split_docs, embedding=embeddings)

No sentence-transformers model found with name MTSAIR/Cotype-Nano. Creating a new one with mean pooling.


In [ ]:
# results = faiss_index.similarity_search(query, k = 10)
# for res in results: 
#     print(f"* {res.page_content}")

In [9]:
pipe = pipeline("text-generation", model="MTSAIR/Cotype-Nano", device="cuda")
# messages = [
#   {"role": "system", "content": f"Ты — ИИ-помощник. Тебе дано задание: необходимо сгенерировать подробный и развернутый ответ. Пользователем является плохо разбирающийся в технологиях человек. \
#   Нужно в несколько абзацев на основе предоставленной информации сформировать для пользователя доступный и понятный ответ. В виде сплошного текста, чтобы пользователь мог понять, с чем имеет дело.\
#   Тебе надо объяснить простому работнику на основе технической документации какие действия следует предпринять для решения его запроса. \
#     Грамотно, коротко и технично опиши основные пункты для решения данного вопроса.\
#     Предоставленная информация должна относиться только к его вопросу, не касаясь любых посторонних тем.\
#     Вторым абзацем составь короткий чек-лист по предоставленным ранее правилам, примерный вид чек-листа: 1)сделать одно;\n \
#     2)сделать другое и т.д.чтобы работник мог понять как решить проблему.\
#     Также обязательно отформатируй текст и дополни его для предельной естественности\n Данные, на основе которых требуется составить ответ: {results}"[:7800]},
#   {"role": "user", "content": f"Запрос пользователя: {query}\n\n"[:256]},
# ]
# res = pipe(messages, max_length=8192)

# print(res[0]['generated_text'][-1]['content'])

Device set to use cuda


In [10]:
# for res in results:
#     # if file_name == f"file_name:'{res.metadata['source']}'_page:{res.metadata['page']}"
#     print(f"file_name:'{res.metadata['source']}'_page:{res.metadata['page']}")

In [11]:
import pandas as pd
df = pd.read_csv("./test.csv")
df_sub=pd.DataFrame()

In [ ]:
for el in df["user_input"]:
    query = el
    results = faiss_index.similarity_search(query, k = 10 )
    messages = [
  {"role": "system", "content": f"Ты — ИИ-помощник. Тебе дано задание: необходимо сгенерировать подробный и развернутый ответ. Пользователем является плохо разбирающийся в технологиях человек. \
  Нужно в несколько абзацев на основе предоставленной информации сформировать для пользователя доступный и понятный ответ. В виде сплошного текста, чтобы пользователь мог понять, с чем имеет дело.\
  Тебе надо объяснить простому работнику на основе технической документации какие действия следует предпринять для решения его запроса. \
    Грамотно, коротко и технично опиши основные пункты для решения данного вопроса.\
    Предоставленная информация должна относиться только к его вопросу, не касаясь любых посторонних тем.\
    Вторым абзацем составь короткий чек-лист по предоставленным ранее правилам, примерный вид чек-листа: 1)сделать одно;\n \
    2)сделать другое и т.д.чтобы работник м/parsed_images/ог понять как решить проблему.\
    Также обязательно отформатируй текст и дополни его для предельной естественности\n Данные, на основе которых требуется составить ответ: {results}"[:7800]},
  {"role": "user", "content": f"Запрос пользователя: {query}\n\n"[:256]},
]
    res_model = pipe(messages, max_length=8192)

    # lst_vec = []

    # for res in results: 
    #     lst_vec.append(res.page_content)
    print(query)
    print("========================================================")
    print(res_model[0]['generated_text'][-1]['content'])
    
    # new_data = pd.DataFrame([{'contexts':f"{lst_vec}", 'response':res_model[0]['generated_text'][-1]['content']}])
    # df_sub = pd.concat([df_sub, new_data], ignore_index=True)
    # df_sub.to_csv('submission2.csv', sep=',', index=False)

    
    

Как настроить норму внесения удобрений на распределителе ZG-TS?
Для настройки нормы внесения удобрений на распределителе ZG-TS выполните следующие шаги:

1. Откройте приложение "Spreader Application Center" (Center).
2. Перейдите в раздел "CMS-T -00001745-G", где будет указано текущее количество необходимого удобрения.
3. Для каждого типа удобрения найдите информацию о рекомендуемых количествах в вашем регионе.
4. Установите выбранные количества в соответствии с инструкциями, которые предоставляет приложение.
5. Сохраните изменения, чтобы они вступили в силу.

Это позволит вам эффективно использовать ресурсы и получить максимальную пользу от ваших агропроизводственных практик.
Какие действия необходимо выполнить перед первым вводом в эксплуатацию ZG-TS?
Перед первым вводом в эксплуатацию терминала ZG-TS вам нужно выполнить следующие шаги:

1. **Получите необходимые инструкции**: Убедитесь, что у вас есть краткое руководство по эксплуатации ZG-TS, которое содержит информацию о том, как 